Observo que UNA vez ha fallado con:
- C = 1, gamma = 10

In [1]:
from sklearn.model_selection import GridSearchCV
from demo_utils.general import get_data
from sklearn.tree import DecisionTreeClassifier
from demo_utils.learning import get_model
import numpy as np
from IPython.display import Markdown as md
from demo_utils.general import SUPPORTED_DATASETS
import pdb

In [2]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [3]:
my_trace = pdb.set_trace

In [4]:
import warnings
warnings.filterwarnings('ignore')

%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [5]:
sampler_name = 'nystroem'

model_params = {
    'model_name': 'logit',
    'sampler_name': 'identity',
    'pca_bool': False,
    'pca_first': True,
    'box_type': 'none'
}
hp_names = {'dt': 'min_impurity_decrease', 'logit': 'C', 'linear_svc': 'C'}
hp_options = {
    'dt': [10**i for i in range(-10, 1)],
    #'logit': [10**i for i in range(-5, 4)],
    'logit': [1],
    #'logit': 1,
    'linear_svc': [10**i for i in range(-5, 4)]
}
testing_dataset = 'digits'
#rbf_gamma_options = [10**i for i in range(-5, 2)]
#nystroem_gamma_options = [10**i for i in range(-5, 2)]
nystroem_gamma_options = [10]
#nystroem_gamma_options = 10
dts_size = 1000

In [6]:
def find_hparams(model_params, options, dts_name):
    n_comp = 500
    prefix = 'model__'
    tunning_params = {prefix + hp_names[model_params['model_name']]: options}
    model = get_model(**model_params)

    if model_params['sampler_name'] == 'rbf':
        tunning_params['sampler__' + 'gamma'] = rbf_gamma_options
        model.set_params(sampler__n_components=n_comp)
    elif model_params['sampler_name'] == 'nystroem':
        tunning_params['sampler__' + 'gamma'] = nystroem_gamma_options
        model.set_params(sampler__n_components=n_comp)
        

    clf = GridSearchCV(model, tunning_params, cv=10, iid=False, verbose=0)
    #clf = GridSearchCV(model, tunning_params, cv=10, iid=False, verbose=0, n_jobs=1)
    
    data = get_data(dts_name, n_ins=dts_size)
    data_train = data['data_train']
    target_train = data['target_train']

    print('a')
    #pdb.set_trace()
    #my_trace()
    clf.fit(data_train, target_train)
    print('b')
    bp = clf.best_params_
    
    #print(model_params['model_name'], key)
    print(model_params['model_name'])
    print(bp)
    print()

In [7]:
def find_hparams_no_cv(model_params, options, dts_name):
    n_comp = 500
    prefix = 'model__'
    tunning_params = {prefix + hp_names[model_params['model_name']]: options}
    model = get_model(**model_params)

    if model_params['sampler_name'] == 'rbf':
        tunning_params['sampler__' + 'gamma'] = rbf_gamma_options
        model.set_params(sampler__n_components=n_comp)
    elif model_params['sampler_name'] == 'nystroem':
        tunning_params['sampler__' + 'gamma'] = nystroem_gamma_options
        model.set_params(sampler__n_components=n_comp)
        

    #clf = GridSearchCV(model, tunning_params, cv=10, iid=False, verbose=10)
    
    model.set_params(**tunning_params)
    
    data = get_data(dts_name, n_ins=dts_size)
    data_train = data['data_train']
    target_train = data['target_train']

    print('a', end=' ')
    #clf.fit(data_train, target_train)
    model.fit(data_train, target_train)
    print('b')
    #bp = clf.best_params_
    
    #print(model_params['model_name'], key)
    #print(model_params['model_name'])
    #print(bp)
    print()

In [8]:
def test_sampler_pca_last(sampler_name):
    display(md('### PCA Last'))
    model_params['pca_bool'] = True
    model_params['pca_first'] = False
    model_params['sampler_name'] = sampler_name
    #for model_name in hp_names:
    model_name = 'logit'
    
    model_params['model_name'] = model_name
    find_hparams(model_params, options=hp_options[model_params['model_name']], dts_name=testing_dataset)
    
def test_simple():
    dt_name = 'mnist'
    display(md('## Simple'))
    model_params['pca_bool'] = False
    model_params['sampler_name'] = 'identity'
    #for model_name in hp_names:
    model_name = 'dt'
    
    model_params['model_name'] = model_name
    print('testing_dataset es {}'.format(testing_dataset))
    find_hparams(model_params, options=hp_options[model_params['model_name']], dts_name=dt_name)

In [9]:
def adv_test_sampler_pca_last(sampler_name):
    '''
    Hace lo mismo que el anterior, pero él mismo hace varias ejecuciones
    '''
    model_params['pca_bool'] = True
    model_params['pca_first'] = False
    model_params['sampler_name'] = sampler_name
    #for model_name in hp_names:
    model_name = 'logit'
    model_params['model_name'] = model_name
    times = 50
    for i in range(times):
        print(i)
        display(md('# {}'.format(i)))
        find_hparams(model_params, options=hp_options[model_params['model_name']], dts_name=testing_dataset)
        print()

In [10]:
def adv_test_sampler_pca_last_no_cv(sampler_name):
    '''
    Hace lo mismo que el anterior, pero él mismo hace varias ejecuciones
    '''
    model_params['pca_bool'] = True
    model_params['pca_first'] = False
    model_params['sampler_name'] = sampler_name
    #for model_name in hp_names:
    model_name = 'logit'
    model_params['model_name'] = model_name
    times = 200
    for i in range(times):
        print(i)
        display(md('# {}'.format(i)))
        #find_hparams(model_params, options=hp_options[model_params['model_name']], dts_name=testing_dataset)
        find_hparams_no_cv(model_params, options=hp_options[model_params['model_name']], dts_name=testing_dataset)
        print()

In [11]:
from sklearn.model_selection import _validation

In [12]:
import sys

In [13]:
path = '/home/hobber/.local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py'
sys.path.append(path)

In [14]:
from sklearn.model_selection._validation import _fit_and_score

In [15]:
sdf

NameError: name 'sdf' is not defined

In [ ]:
%%notify
adv_test_sampler_pca_last_no_cv(sampler_name)

In [ ]:
%%notify
adv_test_sampler_pca_last(sampler_name)

In [ ]:
my_trace = pdb.set_trace

In [ ]:
pdb.run('test_sampler_pca_last(sampler_name)')

In [ ]:
pdb.run('cosa()')

In [18]:
%%notify
cosa()

0


### PCA Last

a
b
logit
{'model__C': 1, 'sampler__gamma': 10}

1


### PCA Last

a
b
logit
{'model__C': 1, 'sampler__gamma': 10}

2


...

a
b
logit
{'model__C': 1, 'sampler__gamma': 10}

113


### PCA Last

a


LinAlgError: SVD did not converge

<IPython.core.display.Javascript object>

In [17]:
def cosa():
    times = 1000
    for i in range(times):
        print(i)
        test_sampler_pca_last(sampler_name)

In [16]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [ ]:
%%notify
# currentwork
times = 100
for i in range(times):
    print(i)
    test_sampler_pca_last(sampler_name)

In [ ]:
%%notify
times = 0
for i in range(times):
    test_simple()

In [ ]:
def _passthrough_scorer(estimator, *args, **kwargs):
    """Function that wraps estimator.score"""
    return estimator.score(*args, **kwargs)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.utils import indexable
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection._validation import _fit_and_score
from itertools import product

In [ ]:
def test_func():
    scorers = {'score': _passthrough_scorer}

    model_params = {
        'model_name': 'logit',
        'sampler_name': 'nystroem',
        'pca_bool': True,
        'pca_first': False,
        'box_type': 'none'
    }

    dts_name = 'digits'
    dts_size = 1000
    data = get_data(dts_name, n_ins=dts_size)
    X = data['data_train']
    y = data['target_train']
    groups = None

    param_grid = {'model__' + 'C': [10**i for i in range(-5, 4)]}
    param_grid = list(ParameterGrid(param_grid))


    cv = KFold(10)
    X, y, groups = indexable(X, y, groups)
    model = get_model(**model_params)
    d = cv.split(X, y, groups)

    fit_and_score_kwargs = dict(scorer=scorers,
                                fit_params=None,
                                return_train_score=True,
                                return_n_test_samples=True,
                                return_times=True,
                                return_parameters=False,
                                error_score='raise-deprecating',
                                verbose=0)

    out = []
    for parameters, (train, test) in product(param_grid, d):
        res = _fit_and_score(estimator=model,
                             X=X,
                             y=y,
                             train=train,
                             test=test,
                             parameters=parameters,
                             **fit_and_score_kwargs)
        out.append(res)
    print('Finished')

In [ ]:
for i in range(500):
    print(i, end=' ')
    test_func()

In [ ]:
from sklearn.model_selection import KFold


In [ ]:
dts_name = 'mnist'
n_splits = 10

In [ ]:
data = get_data(dts_name, n_ins=dts_size)
data_train = data['data_train']
target_train = data['target_train']

In [ ]:
kf = KFold(n_splits=n_splits)

In [ ]:
def test_splits(data, model, model_params, tunning_params, n_splits):
    model = get_model(**model_params)
    model.set_params(**tunning_params)
    kf = KFold(n_splits=n_splits)
    data_train = data['data_train'].values
    target_train = data['target_train'].values
    for train_index, test_index in kf.split(data_train):
        X_train, X_test = data_train[train_index], data_train[test_index]
        y_train, y_test = target_train[train_index], target_train[test_index]
        print('     split: start', end='--')
        model.fit(X_train, y_train)
        sc = model.score(X_test, y_test)
        print('[{}]'.format(sc), end='')
        print('end')

In [ ]:
def find_hparams_no_cv_v6(model_params, options, dts_name):
    n_comp = 500
    prefix = 'model__'
    tunning_params = {prefix + hp_names[model_params['model_name']]: options}
    model = get_model(**model_params)

    if model_params['sampler_name'] == 'rbf':
        tunning_params['sampler__' + 'gamma'] = rbf_gamma_options
        model.set_params(sampler__n_components=n_comp)
    elif model_params['sampler_name'] == 'nystroem':
        tunning_params['sampler__' + 'gamma'] = nystroem_gamma_options
        model.set_params(sampler__n_components=n_comp)
        

    data = get_data(dts_name, n_ins=dts_size)
    test_splits(data, model, model_params, tunning_params, 10)

In [ ]:
def adv_test_sampler_pca_last_no_cv_v6(sampler_name):
    '''
    Hace lo mismo que el anterior, pero él mismo hace varias ejecuciones
    '''
    model_params['pca_bool'] = True
    model_params['pca_first'] = False
    model_params['sampler_name'] = sampler_name
    #for model_name in hp_names:
    model_name = 'logit'
    model_params['model_name'] = model_name
    times = 500
    for i in range(times):
        print(i)
        display(md('# {}'.format(i)))
        #find_hparams(model_params, options=hp_options[model_params['model_name']], dts_name=testing_dataset)
        find_hparams_no_cv_v6(model_params, options=hp_options[model_params['model_name']], dts_name=testing_dataset)
        print()

In [ ]:
%%notify
adv_test_sampler_pca_last_no_cv_v6(sampler_name)